In [1]:
#importing all libraries we will use
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ssl
import re
import json # library to handle JSON files
#!conda install -c conda-forge geotext --yes
from geotext import GeoText

import googlemaps
from datetime import datetime

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="uni_expl")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [57]:
geo=pd.read_csv('GEOCLUSTER.csv')
geo.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
geo.head()

,University,rank,Country,Latitude,Longitude,rank cluster
0,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),1,United States,42.360091,-71.094160,0
1,STANFORD UNIVERSITY,2,United States,37.427475,-122.169719,0
2,HARVARD UNIVERSITY,3,United States,42.377003,-71.116660,0
3,CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH),4,United States,34.137658,-118.125269,0
4,UNIVERSITY OF OXFORD,5,United Kingdom,51.754816,-1.254367,0


In [58]:
venues=pd.read_csv('UNIVENUES.csv')
venues.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
venues.head()

,University,University Latitude,University Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),42.360091,-71.09416,Zesiger Sports And Fitness Center,42.358802,-71.095888,Gym / Fitness Center
1,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),42.360091,-71.09416,Flour Bakery + Cafe,42.361123,-71.096521,Bakery
2,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),42.360091,-71.09416,Central Bottle Wine + Provisions,42.361246,-71.096741,Wine Shop
3,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),42.360091,-71.09416,MIT Johnson Athletics Center (Building W34),42.358856,-71.095932,College Gym
4,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),42.360091,-71.09416,Cafe Luna,42.362875,-71.094341,Café


In [59]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot['University'] = venues['University'] 
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.shape

(33271, 631)

In [60]:
uni_grouped = onehot.groupby('University').mean().reset_index()
uni_grouped.head()

,University,Zoo Exhibit,ATM,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,Alternative Healer,American Restaurant,...,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant,Yunnan Restaurant,Zoo
0,AALBORG UNIVERSITY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AALTO UNIVERSITY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AARHUS UNIVERSITY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ABERYSTWYTH UNIVERSITY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ABO AKADEMI UNIVERSITY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['University']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
uni_venues_sorted = pd.DataFrame(columns=columns)
uni_venues_sorted['University'] = uni_grouped['University']

for ind in np.arange(uni_grouped.shape[0]):
    uni_venues_sorted.iloc[ind, 1:] = return_most_common_venues(uni_grouped.iloc[ind, :], num_top_venues)

uni_venues_sorted.head()

,University,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AALBORG UNIVERSITY,Gym,Bus Station,Zoo,Fabric Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,AALTO UNIVERSITY,Cafeteria,Recreation Center,Bus Stop,Waterfront,Sandwich Place,Event Space,Coffee Shop,Pier,Metro Station,Hotel
2,AARHUS UNIVERSITY,Science Museum,Bar,Park,Student Center,Eye Doctor,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm
3,ABERYSTWYTH UNIVERSITY,Movie Theater,College Quad,Pharmacy,Coffee Shop,Zoo,Fabric Shop,Field,Fast Food Restaurant,Farmers Market,Farm
4,ABO AKADEMI UNIVERSITY,Café,Bar,Restaurant,Scandinavian Restaurant,Gym,Wine Bar,Italian Restaurant,History Museum,Museum,Pub


In [63]:
uni_merged = geo
uni_merged = uni_merged.join(uni_grouped.set_index('University'), on='University')
uni_merged.dropna(inplace=True)
uni_merged.head()

,University,rank,Country,Latitude,Longitude,rank cluster,Zoo Exhibit,ATM,Acai House,Accessories Store,...,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant,Yunnan Restaurant,Zoo
0,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),1,United States,42.360091,-71.094160,0,0.0,0.0,0.0,0.0,...,0.019231,0.0,0.0,0.00000,0.0,0.0,0.019231,0.0,0.0,0.0
1,STANFORD UNIVERSITY,2,United States,37.427475,-122.169719,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0
2,HARVARD UNIVERSITY,3,United States,42.377003,-71.116660,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.01087,0.0,0.0,0.000000,0.0,0.0,0.0
3,CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH),4,United States,34.137658,-118.125269,0,0.0,0.0,0.0,0.0,...,0.050000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0
4,UNIVERSITY OF OXFORD,5,United Kingdom,51.754816,-1.254367,0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0


In [64]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from sklearn import model_selection,datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import preprocessing

In [67]:
X_old=uni_merged.drop(columns=['University','rank','rank cluster','Country','Latitude','Longitude'])


In [68]:
pca = PCA(n_components=3)
X=pca.fit(X_old).transform(X_old)


In [77]:
kclusters = 2

uni_grouped_clustering = X

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(uni_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [80]:
uni_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
uni_merged = geo

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
uni_merged = uni_merged.join(uni_venues_sorted.set_index('University'), on='University')

uni_merged.head() # check the last columns!

,University,rank,Country,Latitude,Longitude,rank cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,MASSACHUSETTS INSTITUTE OF TECHNOLOGY (MIT),1,United States,42.360091,-71.094160,0,1.0,Lounge,Coffee Shop,Rental Car Location,Pizza Place,Sandwich Place,College Gym,Pub,New American Restaurant,Italian Restaurant,Burrito Place
1,STANFORD UNIVERSITY,2,United States,37.427475,-122.169719,0,0.0,Coffee Shop,Café,Fountain,Sculpture Garden,Sandwich Place,Monument / Landmark,Food Truck,Outdoor Sculpture,Grocery Store,Chinese Restaurant
2,HARVARD UNIVERSITY,3,United States,42.377003,-71.116660,0,0.0,Bookstore,Café,Pizza Place,Ice Cream Shop,Gastropub,Mexican Restaurant,Boutique,American Restaurant,Burger Joint,Coffee Shop
3,CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH),4,United States,34.137658,-118.125269,0,0.0,Theater,Café,Park,Scenic Lookout,Hotel,Gym,Gym / Fitness Center,Coffee Shop,Pool,Food & Drink Shop
4,UNIVERSITY OF OXFORD,5,United Kingdom,51.754816,-1.254367,0,1.0,Pub,Coffee Shop,Café,Thai Restaurant,Sandwich Place,Burger Joint,Bookstore,History Museum,Hotel,Art Gallery


In [81]:
uni_merged['Cluster Labels'].replace(to_replace = np.nan, value =float(kclusters),inplace=True) 

In [85]:
uni_merged.loc[uni_merged['Cluster Labels'] == 0.0]

,University,rank,Country,Latitude,Longitude,rank cluster,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,STANFORD UNIVERSITY,2,United States,37.427475,-122.169719,0,0.0,Coffee Shop,Café,Fountain,Sculpture Garden,Sandwich Place,Monument / Landmark,Food Truck,Outdoor Sculpture,Grocery Store,Chinese Restaurant
2,HARVARD UNIVERSITY,3,United States,42.377003,-71.116660,0,0.0,Bookstore,Café,Pizza Place,Ice Cream Shop,Gastropub,Mexican Restaurant,Boutique,American Restaurant,Burger Joint,Coffee Shop
3,CALIFORNIA INSTITUTE OF TECHNOLOGY (CALTECH),4,United States,34.137658,-118.125269,0,0.0,Theater,Café,Park,Scenic Lookout,Hotel,Gym,Gym / Fitness Center,Coffee Shop,Pool,Food & Drink Shop
5,UNIVERSITY OF CAMBRIDGE,6,United Kingdom,52.204267,0.114908,0,0.0,Café,Coffee Shop,Pub,Clothing Store,Burger Joint,Sushi Restaurant,Bar,Sandwich Place,Bookstore,Park
6,ETH ZURICH (SWISS FEDERAL INSTITUTE OF TECHNOL...,7,Switzerland,47.376313,8.547670,0,0.0,Hotel,Swiss Restaurant,Restaurant,Bar,Café,Italian Restaurant,Coffee Shop,Tram Station,Sandwich Place,Bakery
7,IMPERIAL COLLEGE LONDON,8,United Kingdom,51.498800,-0.174877,0,0.0,Café,Hotel,Science Museum,Exhibit,Italian Restaurant,Garden,Ice Cream Shop,Gift Shop,Bakery,Coffee Shop
10,NATIONAL UNIVERSITY OF SINGAPORE (NUS),11,Singapore,1.296643,103.776394,0,0.0,College Cafeteria,Bus Station,Gym / Fitness Center,Chinese Restaurant,Coffee Shop,French Restaurant,Salad Place,Lounge,Asian Restaurant,Performing Arts Venue
11,NANYANG TECHNOLOGICAL UNIVERSITY (NTU),12,Singapore,1.348310,103.683135,0,0.0,Coffee Shop,College Cafeteria,Fast Food Restaurant,Bus Station,Indonesian Restaurant,Soup Place,Sandwich Place,Food Court,Athletics & Sports,Ethiopian Restaurant
12,PRINCETON UNIVERSITY,13,United States,40.343094,-74.655074,0,0.0,Plaza,Sushi Restaurant,Tennis Court,Theater,Art Museum,College Quad,Garden,Hockey Field,New American Restaurant,French Restaurant
13,CORNELL UNIVERSITY,14,United States,42.453449,-76.473503,0,0.0,Tourist Information Center,Convenience Store,College Gym,Scenic Lookout,Garden,Lake,Bowling Alley,Volleyball Court,Café,Farm
